# 📚 RAG Text Chunking Techniques with LangChain

**Text chunking** is a fundamental step in building Retrieval-Augmented Generation (RAG) systems. It involves breaking down large documents into smaller, manageable pieces that can be efficiently stored, retrieved, and processed.

## 🎯 What You'll Learn

1. **Fixed Length Chunking**: Basic character-based splitting
2. **Sliding Window Chunking**: Overlapping chunks for context preservation
3. **Semantic/Structure Chunking**: Paragraph and special character-based splitting
4. **Semantic Chunking with LLM**: AI-powered chunking based on meaning

---

## 📊 Project Overview

**Goal:** Explore different text chunking strategies and understand their trade-offs.

**Key Concepts:**
- 📝 **Chunk Size**: How many characters/words per chunk
- 🔄 **Overlap**: Overlapping sections between chunks
- 🎯 **Separators**: How to split text (paragraphs, sentences, etc.)
- 🧠 **Semantic Boundaries**: Splitting at natural meaning boundaries

---

## ⚙️ Section 1: Setup and Sample Text


In [1]:
sample_text = """Artificial intelligence (AI) is transforming how we interact with technology.
Machine learning, a subset of AI, enables computers to learn from data without
explicit programming. Deep learning, which uses neural networks with multiple
layers, has achieved remarkable results in image recognition, natural language
processing, and game playing. These technologies are now being applied to
real-world problems like medical diagnosis, autonomous vehicles, and climate modeling.


Climate change represents one of the most pressing challenges of our time. Rising
global temperatures are causing sea levels to rise, extreme weather events to become
more frequent, and ecosystems to shift dramatically. Scientists have documented a
clear connection between human activities, particularly the burning of fossil fuels,
and the increase in atmospheric carbon dioxide. Mitigation strategies include
transitioning to renewable energy sources, improving energy efficiency, and
implementing carbon capture technologies.


In the culinary world, fermentation has experienced a renaissance. This ancient
food preservation technique not only extends shelf life but also enhances nutritional
value and creates complex flavors. From kimchi and sauerkraut to kombucha and sourdough
bread, fermented foods are celebrated for their probiotic benefits. The fermentation
process involves beneficial bacteria and yeasts that break down sugars and produce
organic acids, creating the characteristic tangy taste.


The history of the Roman Empire spans over a millennium, from its legendary founding
in 753 BCE to the fall of Constantinople in 1453 CE. At its height, the empire
controlled territories across three continents, implementing sophisticated systems
of governance, engineering, and law. Roman innovations in architecture, including
the arch, concrete, and aqueducts, influenced construction for centuries. The empire's
legacy persists in modern legal systems, languages, and cultural institutions.


Quantum computing represents a paradigm shift in computational technology. Unlike
classical computers that use bits representing either 0 or 1, quantum computers
use qubits that can exist in superposition, representing both states simultaneously.
This property, along with quantum entanglement, enables quantum computers to solve
certain problems exponentially faster than classical computers. Applications include
cryptography, drug discovery, financial modeling, and optimization problems.
"""


In [2]:
print(f"Total characters: {len(sample_text)}")
print(f"Total words: {len(sample_text.split())}")

Total characters: 2478
Total words: 331


## 1. Fixed Length Chunking

In [3]:
from langchain_text_splitters import CharacterTextSplitter

fixed_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0,
    length_function=len,
    separator=""
)

fixed_chunks = fixed_splitter.split_text(sample_text)

print(f"Number of chunks: {len(fixed_chunks)}\n")
for i, chunk in enumerate(fixed_chunks, 1):
    print(f"=== Chunk {i} ({len(chunk)} chars) ===")
    print(chunk)
    print()

Number of chunks: 13

=== Chunk 1 (199 chars) ===
Artificial intelligence (AI) is transforming how we interact with technology.
Machine learning, a subset of AI, enables computers to learn from data without
explicit programming. Deep learning, which

=== Chunk 2 (200 chars) ===
uses neural networks with multiple
layers, has achieved remarkable results in image recognition, natural language
processing, and game playing. These technologies are now being applied to
real-world p

=== Chunk 3 (200 chars) ===
roblems like medical diagnosis, autonomous vehicles, and climate modeling.


Climate change represents one of the most pressing challenges of our time. Rising
global temperatures are causing sea level

=== Chunk 4 (200 chars) ===
s to rise, extreme weather events to become
more frequent, and ecosystems to shift dramatically. Scientists have documented a
clear connection between human activities, particularly the burning of fos

=== Chunk 5 (200 chars) ===
sil fuels,
and the increase in 

## 2. Fixed Length + Sliding Window

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

window_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # Same size
    chunk_overlap=80,  # Larger overlap = bigger window
    length_function=len,
    separators=[""]  # No special separators, pure sliding window
)

window_chunks = window_splitter.split_text(sample_text)

print(f"Number of chunks: {len(window_chunks)}\n")

# Limit to first 5 chunks
for i, chunk in enumerate(window_chunks[:5], 1):
    print(f"=== Chunk {i} ({len(chunk)} chars) ===")
    print(chunk)

    # Show overlap with previous chunk
    if i > 1:
        prev_chunk = window_chunks[i - 2]
        overlap_text = ""
        for j in range(min(len(prev_chunk), 100), 0, -1):
            if prev_chunk[-j:] == chunk[:j]:
                overlap_text = chunk[:j]
                break
        if overlap_text:
            print(f"\n[Overlap with previous: {len(overlap_text)} chars]")
    print()


Number of chunks: 20

=== Chunk 1 (199 chars) ===
Artificial intelligence (AI) is transforming how we interact with technology.
Machine learning, a subset of AI, enables computers to learn from data without
explicit programming. Deep learning, which

=== Chunk 2 (200 chars) ===
computers to learn from data without
explicit programming. Deep learning, which uses neural networks with multiple
layers, has achieved remarkable results in image recognition, natural language
proces

[Overlap with previous: 79 chars]

=== Chunk 3 (200 chars) ===
s, has achieved remarkable results in image recognition, natural language
processing, and game playing. These technologies are now being applied to
real-world problems like medical diagnosis, autonomo

[Overlap with previous: 80 chars]

=== Chunk 4 (200 chars) ===
es are now being applied to
real-world problems like medical diagnosis, autonomous vehicles, and climate modeling.


Climate change represents one of the most pressing challenges of our time.

## 3. Paragraph / Special Character Chunking

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

paragraph_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    length_function=len,
    separators=["\n\n"]
)

paragraph_chunks = paragraph_splitter.split_text(sample_text)

print(f"Number of chunks: {len(paragraph_chunks)}\n")
for i, chunk in enumerate(paragraph_chunks, 1):
    print(f"=== Chunk {i} ({len(chunk)} chars) ===")
    print(chunk)
    print()


Number of chunks: 5

=== Chunk 1 (474 chars) ===
Artificial intelligence (AI) is transforming how we interact with technology.
Machine learning, a subset of AI, enables computers to learn from data without
explicit programming. Deep learning, which uses neural networks with multiple
layers, has achieved remarkable results in image recognition, natural language
processing, and game playing. These technologies are now being applied to
real-world problems like medical diagnosis, autonomous vehicles, and climate modeling.

=== Chunk 2 (532 chars) ===



Climate change represents one of the most pressing challenges of our time. Rising
global temperatures are causing sea levels to rise, extreme weather events to become
more frequent, and ecosystems to shift dramatically. Scientists have documented a
clear connection between human activities, particularly the burning of fossil fuels,
and the increase in atmospheric carbon dioxide. Mitigation strategies include
transitioning to renewable energ

## 4. Semantic Chunking with LLM

In [6]:
!pip install -q langchain_experimental langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [8]:
import os
from google.colab import userdata
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import nltk

# Setup
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_1')
nltk.download('gutenberg', quiet=True)

from nltk.corpus import gutenberg

book_text = gutenberg.raw('austen-emma.txt')

long_text = book_text[:5000]

print(f"Book: Emma by Jane Austen")
print(f"Text length: {len(long_text)} characters")

Book: Emma by Jane Austen
Text length: 5000 characters


https://python.langchain.com/docs/how_to/semantic-chunker/


The default way to split is based on percentile. In this method, all differences between sentences are calculated, and then any difference greater than the X percentile is split. The default value for X is 95.0 and can be adjusted by the keyword argument breakpoint_threshold_amount which expects a number between 0.0 and 100.0.



High percentile (closer to 100) → splitter is very strict: only big drops in similarity trigger splits → fewer chunks.

Low percentile (closer to 0) → splitter is very sensitive: even small drops trigger splits → many chunks.

In [12]:
# Semantic chunking with percentile
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
semantic_splitter = SemanticChunker(
    gemini_embeddings,
    breakpoint_threshold_type="percentile", # places where similarity drops below a threshold.
    breakpoint_threshold_amount=80, # Parameter to play with 0-100
)

chunks = semantic_splitter.split_text(long_text)

print(f"Number of semantic chunks: {len(chunks)}")

# Print full chunks
for i, chunk in enumerate(chunks, 1):
    print(f"{'='*70}")
    print(f"CHUNK {i} ({len(chunk)} characters)")
    print(f"{'='*70}")
    print(chunk)
    print(f"\n{'='*70}\n")

Number of semantic chunks: 7
CHUNK 1 (476 characters)
[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her. She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.


CHUNK 2 (223 characters)
Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.


CHUNK 3 (939 characters)
Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma. Between _them_ it was more the intimacy